In [3]:
!conda install -c conda-forge librosa -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - librosa


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    appdirs-1.4.3              |             py_1          11 KB  conda-forge
    audioread-2.1.8            |   py38h32f6830_2          32 KB  conda-forge
    ffmpeg-4.3.1               |       h167e202_0        83.3 MB  conda-forge
    gettext-0.19.8.1           |    hc5be6a0_1002         3.6 MB  conda-forge
    gnutls-3.6.13              |       h79a8f9a_0         2.1 MB  conda-forge
    lame-3.100                 |    h14c3975_1001         498 KB  conda-forge
    libflac-1.3.3              |       he1b5a44_0         517 KB  conda-forge
    libiconv-1.15

In [1]:
%matplotlib widget
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [2]:
import asyncio

class Timer:
    def __init__(self, timeout, callback):
        self._timeout = timeout
        self._callback = callback
        self._task = asyncio.ensure_future(self._job())

    async def _job(self):
        await asyncio.sleep(self._timeout)
        self._callback()

    def cancel(self):
        self._task.cancel()
        
def debounce(wait):
    def decorator(fn):
        timer = None
        def debounced(*args, **kwargs):
            nonlocal timer
            def call_it():
                fn(*args, **kwargs)
            if timer is not None:
                timer.cancel()
            timer = Timer(wait, call_it)
        return debounced
    return decorator


In [7]:
def update_plot(duration):
    ax[0].cla()
    ax[1].cla()
    y, sr = librosa.load(librosa.ex('choice'), duration = duration)
    D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
    librosa.display.specshow(D, y_axis='linear', x_axis='time', sr=sr, ax=ax[0])
    return librosa.display.specshow(D, y_axis='log', x_axis='time', sr=sr, ax=ax[1])    

@debounce(1.0)
def handle_slider_change(change):
    duration = change.new
    caption.value = 'Spectrogram duration: {} secs.'.format(duration)
    update_plot(duration)

caption = widgets.Label(value="Spectrogram duration: 5 secs.")
slider = widgets.IntSlider(min = 5, max = 100, step = 5)
slider.observe(handle_slider_change, names='value')

display(caption, slider)

fig, ax = plt.subplots(nrows=2, ncols = 1, sharex = True, figsize=(14,8))
ax[0].set(title="Linear Frequency spectrogram")
ax[1].set(title="Log Frequency spectrogram")
ax[1].label_outer()
fig.colorbar(update_plot(5), ax=ax, format="%+2.f dB");
plt.clf()


Label(value='Spectrogram duration: 5 secs.')

IntSlider(value=5, min=5, step=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
def update_plot(duration):
    y, sr = librosa.load(librosa.ex('choice'), duration = duration)
    y_harm, y_perc = librosa.effects.hpss(y)
    titles = ["Sound wave plot", "Harmonic And Percursive plot"]
    alphas = [1.0, 0.5]
    ys = [y, y_harm]
    for i in range(len(titles)):
        axs[i].cla()
        axs[i].set(title=titles[i])
        axs[i].label_outer()
        librosa.display.waveplot(ys[i], sr=sr, ax=axs[i], alpha = alphas[i])
    librosa.display.waveplot(y_perc, sr=sr, color='r', alpha=0.5, ax=axs[1])

@debounce(1.0)
def handler_duration_slider(change):
    duration = change.new
    caption.value = "Song duration: {} secs".format(duration)
    update_plot(duration)    
    
caption = widgets.Label(value="Song duration: 5 secs")
slider = widgets.IntSlider(value = 5, min = 1, max = 50, step = 1)
slider.observe(handler_duration_slider, names='value')

display(caption, slider)

fig, axs = plt.subplots(nrows=2, sharex=True, sharey=True);
update_plot(5)

Label(value='Song duration: 5 secs')

IntSlider(value=5, max=50, min=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …